In [1]:
from collections import Counter, defaultdict
import heapq

In [3]:
# Task Scheduler 

# You are given an array of CPU tasks, each represented by letters A to Z, and a cooling time, n.
# Each cycle or interval allows the completion of one task. Tasks can be completed in any order, but there's a constraint:
#                                           identical tasks must be separated by at least n intervals due to cooling time.
# Return the minimum number of intervals required to complete all tasks.


'''
# Interface
Args:
    tasks: ["A", "C", "B"]
    n: cooling period
def min_intervals_of_completion(tasks, n) -> return num intervals


# Example

(1) AAB, n = 1
BA-A 
ABA ---> return 3

(2) AABBBC,n = 3
rem = B1
BAC-BA--B 

# Algorithm
(1)
Select each char greedy every time.
With heap. # (-cnt, char), next_possible_t
   - I find the task most frequent in each t.
   - 

AAABBC n = 3

(-3,A,1), (-2,B,1), (-1,C,1)
t = 1, get (-3, A)
(-2,B), (-1,C) <---- (-2, A, 5)
t = 2, get (-2, A)

(2)
I will keep track
- heap # (- cnt, task_id)
- tasks_in_cooling_period = {available_time -> task_id[] }


for t = 0~,
    - add tasks of tasks_in_cooling_period[t]
    - pop out one task.
    - add the task to tasks_in_cooling_period

aaaa, n = 10000

time = N * max(tasks) + N logN
space = N
'''

# Impl
def min_intervals_of_completion(tasks, n): # "ABAB", 10
    task_cnt_and_type = [(-cnt, task_type) for task_type, cnt in Counter(tasks).items()] # (-2,A), (-2, B) #[](-1, A)
    heapq.heapify(task_cnt_and_type)
    tasks_in_cooling_period = defaultdict(list) # {task_type -> (-cnt, task_type)} # {} -> {12: (-1, A), 13:(-1, B)}

    t = 0 # 2->3-> 13
    while 0 < len(task_cnt_and_type) or 0 < len(tasks_in_cooling_period):
        t += 1
        for task in tasks_in_cooling_period[t]:
            heapq.heappush(task_cnt_and_type, task)

        ### I forgot to delete this !!!!!!!!!!!
        ### Without this, it loops inifiitely due to `or 0 < len(tasks_in_cooling_period)`
        if t in tasks_in_cooling_period:
            del tasks_in_cooling_period[t]

        if len(task_cnt_and_type) == 0:
            continue

        out = heapq.heappop(task_cnt_and_type)
        cnt, task_type = - out[0], out[1] # 1,  B

        # task is done here.
        rem_cnt = cnt - 1 # 0
        if 1 <= rem_cnt:
            tasks_in_cooling_period[t + n + 1].append((-rem_cnt, task_type))

    return t


# Test

assert min_intervals_of_completion("ABAB", 10) == 13
assert min_intervals_of_completion("AA", 10) == 12
assert min_intervals_of_completion("", 10) == 0 # pass

In [ ]:
# Task Scheduler I (2nd trial)
'''
# interface
Args:
    tasks: like [1,2,1] (array of positive integers)
    space: like 5, positive int
def min_days_completion(tasks, space) -> return min num of days.

- if len(tasks) == 0 -> return 0

# example

tasks: [1,2,1]
space: 5

1 2 - - - 1  -------> return 6.

# algorithm

On each day, do tasks which is remains the most && possible. (in greedy way)
-> Use heap (-cnt, task_id)

day = 0
on while 0 < len(heap):
    day += 1
    move availble_from[day] to heap.
    pop from heap.
    decrement cnt and add to availble_from[day + space]

return day

e.g. [11,22,11], space = 5
-> {11: 2, 22: 1}
'''

# impl
def min_days_completion(tasks, space): # [1,1,2], 3
    # optional optimization.
    # if space == 0:
    #     return len(tasks)

    rem_tasks = [(-cnt, task_id) for task_id, cnt in Counter(tasks).items()] # x(-2,1), x(-1,2), x(-1,1)
    heapify(rem_tasks)
    
    available_from = defaultdict(list) # {}-> 4:(-1,1) -> {}

    curr_day = 0 # 0->1->2->3->4

    while 0 < len(rem_tasks) or 0 < len(available_from):
        curr_day += 1
        if curr_day in available_from:
            for revived in available_from[curr_day]:
                heappush(rem_tasks, revived)
            del available_from[curr_day]

        # I forgot this !!!!!!!!!!!!!!!!!
        if len(rem_tasks) == 0:
            continue

        out = heappop(rem_tasks)
        rem_cnt, task_id = - out[0], out[1] # 2,1/ 1,2/ 1,1
        # do task_id
        rem_cnt -= 1 # 1 /0 /0
        if 1 <= rem_cnt:
            available_from[curr_day + space + 1].append((-rem_cnt, task_id))

    return curr_day

# test
# with break to wait for space
assert min_days_completion([1,1,2], 2) == 4 # (with 1->?-?->1 order)
# without break to wait for space
assert min_days_completion([1,1,2], 1) == 3 # (with 1->2->1 order) ✅
# no space
assert min_days_completion([1,1,2], 0) == 3 # (with any order) ✅
# no task
assert min_days_completion([], 3) == 0 # ✅

In [ ]:
# Maximum Number of Events That Can Be Attended

# You are given an array of events where events[i] = [startDayi, endDayi]. Every event i starts at startDayi and ends at endDayi.
# You can attend an event i at any day d where startTimei <= d <= endTimei. You can only attend one event at any time d.
# Return the maximum number of events you can attend.

'''

# Example

[1,5],[1,5],[1,5],[2,3],[2,3]

1------5
1------5
1------5
  2-3
  2-3


curr_day = 1



sort events

heap = [] # (end_date, start_date)

for each day,
   for each event which starts on that day, push it into heap
   pop out event
       if it is invalid (out.end_date < curr.start_date), ignore it.
       otherwise, cnt += 1
'''

from heapq import heappush, heappop, heapify

# Impl
def attendable_cnt(events):
    events.sort()
    cnt = 0
    
    started_events = [] # (end, start)

    i = 0
    curr_day = 1
    while i < len(events) or 0 < len(started_events):
        # Push all the events started today into the heap `started_events`.
        while i < len(events) and events[i][0] <= curr_day:
            event = events[i]
            heappush(started_events, (event[1], event[0]))
            i += 1

        # Pop out all the outdated events from heap.
        while 0 < len(started_events) and started_events[0][0] < curr_day:
            heappop(started_events)

        # Pop out an event attended today.
        if 0 < len(started_events):
            heappop(started_events)
            cnt += 1

        curr_day += 1
    return cnt


'''
time  = NlogN + max(endDate)
space = N

'''

# Test
print(attendable_cnt([[2,2],[1,2], [1,2]]))
assert attendable_cnt([[2,2],[1,2], [1,2]]) == 2
assert attendable_cnt([[2,2],[1,2]]) == 2
assert attendable_cnt([]) == 0
assert attendable_cnt([[1,5],[1,5],[1,5],[2,3],[2,3]]) == 5

In [ ]:
# Reorganize string

'''
aab -> aba
{a: 2, b: 1}
-> use "a" (the most cnt)
{a: 1, b: 1}
-> use "b" (not a)
{a: 1, b: 0}
-> use "a" (the most cnt)

Use min heap.
h = [] # (- cnt, char)

count each char.
build heap (-cnt, char)

keep track of chars = []
while 0 < len(heap)
    pop out from heap(1)
    if out_char != prev_char,
        decrement cnt and push it back.
        append to chars
    otherwise,
        pop one more(2). if there is not any, return ""
        append (2) to chars
        push back (1) and cnt-decrmented (2)

joined chars

Assuming N=len(s) and K=nunique(s),
time  = N log K
space = K

'''

class Solution:
    def reorganizeString(self, s: str) -> str:
        # (-rem_cnt, char)
        rem_chars = [(-cnt, char) for char, cnt in Counter(s).items()]
        heapq.heapify(rem_chars)

        selected_chars = []

        while 0 < len(rem_chars):
            out = heapq.heappop(rem_chars)
            rem_cnt, char = - out[0], out[1]

            if len(selected_chars) == 0 or selected_chars[-1] != char:
                selected_chars.append(char)
                rem_cnt -= 1
                if 0 < rem_cnt:
                    heapq.heappush(rem_chars, (-rem_cnt, char))
            else:
                if len(rem_chars) == 0:
                    return ""
                out2 = heapq.heappop(rem_chars)
                rem_cnt2, char2 = - out2[0], out2[1]
                assert selected_chars[-1] != char2

                selected_chars.append(char2)

                heapq.heappush(rem_chars, (-rem_cnt, char))

                rem_cnt2 -= 1
                if 0 < rem_cnt2:
                    heapq.heappush(rem_chars, (-rem_cnt2, char2))
        
        return "".join(selected_chars)



In [20]:
# Longest Happy Strings

# A string s is called happy if it satisfies the following conditions:

# s only contains the letters 'a', 'b', and 'c'.
# s does not contain any of "aaa", "bbb", or "ccc" as a substring.
# s contains at most a occurrences of the letter 'a'.
# s contains at most b occurrences of the letter 'b'.
# s contains at most c occurrences of the letter 'c'.
# Given three integers a, b, and c, return the longest possible happy string. If there are multiple longest happy strings, return any of them.
# If there is no such string, return the empty string "".

# A substring is a contiguous sequence of characters within a string.

'''
# interface
def longeset_happy_string(a, b, c) -> return string

- a, b, c is integer. 0 <= a,b,c.

# example
a,b,c = 2,3,4

aabbbcccc   x
aabccbbcc   o

a,b,c = 2,2,100
cc a cc b cc a cc b cc
{a2b2c99}-> {a2b2c98} -> {a1b2c98}
cc a cc

# algorithm

Track
- second_last_char, last_chars
- rem_char_to_cnt (list of (- frequency, char))
- chars

while len(rem_char_to_cnt):
    pop element. 
    if out = second_last_char = last_chars, pop again.
    append used one to char.
    decremnt frequency and push it back.
    for the other, push it back
return contanated chars.

time  = A + B + C
space = A + B + C
'''
# Impl
def longeset_happy_string(a, b, c): # a8, b1, c0
    # x(-8, a), (-1, b), (-7, a)
    rem_chars = [(-cnt, char) for char, cnt in [("a", a), ("b", b), ("c", c)] if 0 < cnt] # x(-8, a), (-1, b), x(-7, a),  (-6, a), ()
    heapq.heapify(rem_chars)
    chars = [] # a, a, b
    second_last_char, last_char = None, None # a, a

    while 0 < len(rem_chars):
        out1 = heapq.heappop(rem_chars)
        cnt1, char1 = - out1[0], out1[1] # 6, a

        #  
        if second_last_char == last_char == char1:
            if len(rem_chars) == 0:
                break
            out2 = heapq.heappop(rem_chars)
            cnt2, char2 = - out2[0], out2[1] # 1, b

            chars.append(char2)
            heapq.heappush(rem_chars, (-cnt1, char1))
            second_last_char, last_char = last_char, char2
            if 2 <= cnt2:
                heapq.heappush(rem_chars, (-cnt2+1, char2))
        else:
            chars.append(char1)
            if 2 <= cnt1:
                heapq.heappush(rem_chars, (-cnt1+1, char1))
            second_last_char, last_char = last_char, char1

    return "".join(chars)
    
# Test
print(longeset_happy_string(8,1,1))
assert longeset_happy_string(8,1,1) in ["aabaacaa", "aacaabaa"]
assert longeset_happy_string(8,1,0) == "aabaa"
assert longeset_happy_string(8,0,0) == "aa"
assert longeset_happy_string(0,0,0) == ""

aabaacaa


In [22]:
# Maximum Number of Weeks for Which You Can Work

# There are n projects numbered from 0 to n - 1.
# You are given an integer array milestones where each milestones[i] denotes the number of milestones the ith project has.

# You can work on the projects following these two rules:

# Every week, you will finish exactly one milestone of one project. You must work every week.
# You cannot work on two milestones from the same project for two consecutive weeks.
# Once all the milestones of all the projects are finished, or if the only milestones that you can work on will cause you to violate the above rules, you will stop working.
# Note that you may not be able to finish every project's milestones due to these constraints.

# Return the maximum number of weeks you would be able to work on the projects without violating the rules mentioned above.


'''
# Interface
n projects
milestones (length n)

def max_num_workable_weeks(milestones) -> return int

# Example

[*2] ->
[x1]     ------------------> return 1


[*3,2] ->
[2,*1] ->
[*1,0] ->
[0,0]   -------------------> return 3


# Algorithm

Do the project which has the most ms every week. (greedy)

## BF
time  = len(n) * sum(n)
space = len(n)


## Optimized
Use heap # (- rem_cnt, project_id)

prev_project

week = 0
while 0 < len(heap):
    week += 1
    pop from heap
    do project                             <-------- if this is the prev_project, ignore pop one more (if there isn't break).
    decrement cnt and push it back to heap <-------- push back out1 as it is and out2 decrmented


time  = sum(n) * log len(n)
space = len(n)
'''

## Impl

def max_num_workable_weeks(milestones): # [2,2] #[2]
    project_cnts = [(-ms, project_id) for project_id, ms in enumerate(milestones)] # x(-2,0),x(-2,1), x(-1,0), (-1,1)  ## (-2,0), (-1,0)
    heapify(project_cnts)

    week = 0 # 2=>3->4 ## -> 1
    prev_project_id = None # 0->1->0 ## 0

    while 0 < len(project_cnts):
        out1 = heappop(project_cnts)
        ms1, p1 = - out1[0], out1[1] # 2,0 -> 2,1 -> 1,0 -> 1,1 ->1,0

        if p1 != prev_project_id:

            # do p1
            prev_project_id = p1

            ms1 -= 1 # 1
            if 1 <= ms1:
                heappush(project_cnts, (-ms1, p1))
        else:
            if len(project_cnts) == 0:
                break

            out2 = heappop(project_cnts)
            ms2, p2 = - out2[0], out2[1]

            # do p2
            prev_project_id = p2

            ms2 -= 1
            if 1 <= ms2:
                heappush(project_cnts, (-ms2, p2))
            heappush(project_cnts, (-ms1, p1))

        week += 1
    return week
            
## Test
# assert max_num_workable_weeks([4,1]) == 1

assert max_num_workable_weeks([2,2]) == 4
assert max_num_workable_weeks([2,1]) == 3
assert max_num_workable_weeks([2]) == 1

assert max_num_workable_weeks([]) == 0 # pass

In [25]:
# 2335. Minimum Amount of Time to Fill Cups

# You have a water dispenser that can dispense cold, warm, and hot water.
# Every second, you can either fill up 2 cups with different types of water, or 1 cup of any type of water.

# You are given a 0-indexed integer array amount of length 3 where
#                 amount[0], amount[1], and amount[2] denote the number of cold, warm, and hot water cups you need to fill respectively.
# Return the minimum number of seconds needed to fill up all the cups.

'''
# interface
def min_seconds_to_fill(amount) -> return int

# exmple

*3,*4,5
-> *2,*3,5
-> *1,*2,5
-> 0,*1,*5
-> 0,0,*4
-> 0,0,*3
-> 0,0,*2
-> 0,0,*1
-> 0,0,0       -> return 8


# algorithm

[1,2],8
0,[1,8]
0,[0,7]


## sorting -> Wrong.

least, mid, most

cnt = 0
least_and_mid = least
mid_and_max = mid - least_and_mid
max_only = most - mid_and_max

return least_and_mid + mid_and_max + max_only
total = sum(amt) = 11

# Greedily pick 2 max cups.

time  = sum(amt) * log(len(amt))
space = len(amt)
'''

def min_seconds_to_fill(amount):# [2,1,0]
    rems = [(-amt, type) for type, amt in enumerate(amount) if 0 < amt] # x(-2,0), x(-1,1), x(-1,0)
    heapify(rems) # (-1,0)
    cnt = 0

    while 0 < len(rems):
        out1 = heappop(rems) # (-1,0)
        rem1, type1 = - out1[0], out1[1] # 1,0

        has_one_more_type = 0 < len(rems)

        if has_one_more_type: # true
            out2 = heappop(rems) # 
            rem2, type2 = - out2[0], out2[1] # 1,1
            # poor type1 and type2
            rem1 -= 1 # 1
            if 0 < rem1: heappush(rems, (-rem1, type1))
            rem2 -= 1 # 0
            if 0 < rem2: heappush(rems, (-rem2, type2))
        else:
            # poor type1 only
            rem1 -= 1
            if 0 < rem1: heappush(rems, (-rem1, type1))
            
        cnt += 1
    return cnt
            
            


assert min_seconds_to_fill([5,10,20]) == 20
assert min_seconds_to_fill([0,10,20]) == 20
assert min_seconds_to_fill([0,0,0]) == 0

In [39]:
# Rearrange String k Distance Apart

# Given a string s and an integer k, rearrange s such that the same characters are at least distance k from each other.
# If it is not possible to rearrange the string, return an empty string "".

'''
# interface
Arg:
    s (str)
    k (int)
def rearranged_string(s, k) -> return str (maybe "")

# example
abbccc, k = 3

{a1>0,b2>1>0,c3>2>1}
cbacb
cbacb   c remaing ""

Not possible (c--c--c)


# Algorithm
Choose char which remains the most in greedy

abbccc
heap = [(-3,c), (-2,b), (-1,c)] # (-rem_cnt, char)
idx_to_reviving_chars = {c: 3}

for each index,
    push idx_to_reviving_chars[idx] back into heap
    pop out from heap ### if this is empty, break
    append it to char
    add it to idx_to_reviving_chars[idx+k]

return concatanated chars


assuming N = len(s), K = unique(s),

time = NlogK + N = NlogK <---- ###### +KlogK for heap initialization. -> (N + K) logK
space = K + K = K
'''
# Impl
def rearranged_string(s, k): # aab, k=2
    if k == 0:
        return s

    rem_chars = [(-cnt, char) for char, cnt in Counter(s).items()] # x(-2,a),x(-1,b), (-1,a)
    heapify(rem_chars)

    idx_to_reviving_chars = defaultdict(list) # {idx -> (-cnt, char)} #{2->(-1,a)}

    rearranged_chars = [] # a,b,a

    for idx in range(len(s)): # 0,1,*2
        for reviver in idx_to_reviving_chars[idx]: # (-1,a)
            heappush(rem_chars, reviver)

        if idx in idx_to_reviving_chars:
            del idx_to_reviving_chars[idx]

        #### This cannot be deleted even after optimzation !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        if len(rem_chars) == 0:
            return ""

        out = heappop(rem_chars)
        rem_cnt, curr_char = - out[0], out[1] # 1,a

        rearranged_chars.append(curr_char)
        rem_cnt -= 1 # 1 -> 0

        if 1 <= rem_cnt:
            revival_idx = idx + k # 2
            cannot_revive = len(s) - 1 < revival_idx

            # This is optimization.
            if cannot_revive:
                return ""

            idx_to_reviving_chars[idx + k].append((-rem_cnt, curr_char)) # (-1,a)

    return "".join(rearranged_chars)            
        
# Test
assert rearranged_string("aab", 3) == ""
assert rearranged_string("aab", 2) == "aba"
assert rearranged_string("aab", 1) in ["aab", "aba", "bba"]
assert rearranged_string("aab", 0) in ["aab", "aba", "bba"]

# s in empty
assert rearranged_string("", 1) == ""
assert rearranged_string("", 1) == ""

# I missed this case in LC.
assert rearranged_string("aaa", 2) in ""

In [68]:
# Top K Frequent Words (skipped time optimization)

# You are given two integer arrays nums1 and nums2 sorted in non-decreasing order and an integer k.

# Define a pair (u, v) which consists of one element from the first array and one element from the second array.

# Return the k pairs (u1, v1), (u2, v2), ..., (uk, vk) with the smallest sums.

'''
# Interface
Args:
    if len(nums2) * len(nums2) < k -> return None
def k_smallest_pairs(nums1, nums2, k) -> return array with k elements like [(11,22)...]


# Example
k = 3
nums1 = [1,3,6,7]
nums2 = [3,4,6,9]

products = 4 * 4 pairs.
candidates are 7*9, 7*6, 9*6,,,,

# Algorithm (1) 
sort nums1 in ascending order.
sort nums2 in ascending order.
Use 2 pointers

[1,3,6,7]
 ^p1
[3,4,6,9]
 ^p2

for k times:
    push 7+9
    compare nums1[p1+1] + nums[p2]  vs  nums1[p1] + nums2[p2+_1]  -> move p1 or p2 for smaller one.

time  = NlogN + MlogM + k
space = N + M
 

# Algorithm (2) - top k heap

Use min heap of length k.
for each product, 
    push to heap
    pop smallest if k < len(heap)

time  = NM * logk
space = k
'''
# impl
def k_smallest_pairs(nums1, nums2, k):
    if len(nums1) * len(nums2) < k:
        return None

    smallests = []
    for num1 in nums1:
        for num2 in nums2:
            pair_sum = num1 + num2
            heapq.heappush(smallests, (pair_sum, num1, num2))
            if k < len(smallests):
                heapq.heappop(smallests)
    assert len(smallests) == k

    return list(map(lambda e: (e[1], e[2]), smallests))

# test
# print(k_smallest_pairs([1,2], [1,5], 2))
assert k_smallest_pairs([1,2], [1,5], 2) == [(1, 5), (2, 5)]
assert k_smallest_pairs([1,2], [1,5], 5) is None

In [116]:
# The median is the middle value in an ordered integer list. If the size of the list is even, there is no middle value.
# So the median is the mean of the two middle values.

# For examples, if arr = [2,3,4], the median is 3.
# For examples, if arr = [1,2,3,4], the median is (2 + 3) / 2 = 2.5.
# You are given an integer array nums and an integer k.
# There is a sliding window of size k which is moving from the very left of the array to the very right.
# You can only see the k numbers in the window. Each time the sliding window moves right by one position.

# Return the median array for each window in the original array.

'''

# interface
Args:
    1 <= k othewise return None
    if nums < k -> return []
def sliding_medians(nums, k) -> return array of medians (length = len(nums) - k + 1)


# example
k = 3
5 1 2 4 9
*---*
  *---*
    *---*


1,2,5  ------------> med = 2
add 4, remove 5
1,2,4  ------------> med = 2
add 9, remove 1
2,4,9  ------------> ned = 4


# algorighm - BF
time  = N K logK
space = K

# algorithm - 2 heaps
- smaller_half (max heap)
- larger_half  (min heap)

constraints 
(1) - sh[0] <= lh[0]
(2) len(sh) <= len(lh) <= len(sh) + 1
    That is, 
    len(sh) - sum(sh_invalid_cnts.values()) <= len(lh) - sum(lh_invalid_cnts.values()) <------------ TODO: optimze time later




nums = 7
k = 3

sh = [1]
sh_invalid_cnts = {}
lh = [2,5] 
lh_invlid_cnts = {}

1,2,5  ------------> med = 2

remove 5
add 4 to sh.

Check sh and lh and pop out invalidated one if it is at top.
compare - sh[0] vs lh[0] and rebalance for (1)

Do balancing for (2).

take a look at sh[0] and lh[0] to find median.

sh = [1] -> [1,4] -> [1]
sh_invalid_cnts = {}
lh = [2,x5] -> [2,4,x5]
lh_invlid_cnts = {} -> {5: 1}



1,2,4  ------------> med = 2


add 9, remove 1
2,4,9  ------------> ned = 4

medians  = []

left, right = 0, k - 1
build lh and sh
append median to medians

left = 0
for right in range(k, len(nums)): # r = k ~ len(nums) -
    # remove left
    left += 1
    # add right

    # balance for (1)
    # balance for (2)

    append to medians
return medians


time  = log(N/2) * (N - k + 1) = NlogN (~= NlogK)
space = N (~= K)
'''

from heapq import heappush

# impl
def sliding_medians(nums, k):
    if k == 0 or len(nums) < k:
        return None

    medians = []
    sh = [] # smaller half. used as max heap
    lh = [] # larger half.  used as min heap
    for i in range(k):
        heappush(sh, - nums[i])
        # meet constraint (2)
    while len(sh) > len(lh):
        out = - heappop(sh)
        heappush(lh, out)
    assert len(sh) <= len(lh) <= len(sh) + 1

    sh_invalids = defaultdict(int)
    lh_invalids = defaultdict(int)
        
    medians.append(get_median(sh, sh_invalids, lh, lh_invalids))
    
    left = 0
    for right in range(k, len(nums)): # r = k ~ len(nums)
        # THis has to be after move left
        # # move right
        # heappush(sh, - nums[right])
        # print("added right=", nums[right])

        # move left
        refresh_sh_top(sh, sh_invalids)
        refresh_sh_top(lh, lh_invalids)

        prev_left_num_in_sh = nums[left] <= - sh[0]    
        if prev_left_num_in_sh:
            sh_invalids[nums[left]] += 1
        else:
            lh_invalids[nums[left]] += 1

        print("removed left=", nums[left])
        left += 1

        refresh_sh_top(sh, sh_invalids)
        refresh_sh_top(lh, lh_invalids)
        
        # move right
        heappush(sh, - nums[right])
        print("added right=", nums[right])

        refresh_sh_top(sh, sh_invalids)
        refresh_lh_top(lh, lh_invalids)
    
        # balance heaps for (1)
        if 1 <= len(sh) and 1 <= len(lh) and - sh[0] > lh[0]:
            out = - heappop(sh)
            heappush(lh, out)
    
        refresh_sh_top(sh, sh_invalids)
        refresh_lh_top(lh, lh_invalids)
    
        # balance heaps for (2)
        # sh_valid_cnt <= lh_valid_cnt <= sh_valid_cnt + 1
        sh_valid_cnt = len(sh) - sum(sh_invalids.values())
        lh_valid_cnt = len(lh) - sum(lh_invalids.values())
    
        if lh_valid_cnt < sh_valid_cnt:
            out = - heappop(sh)
            heappush(lh, out)
        if sh_valid_cnt + 1 < lh_valid_cnt:
            out = heappop(lh)
            heappush(sh, - out)
    
        refresh_sh_top(sh, sh_invalids)
        refresh_lh_top(lh, lh_invalids)
    
        medians.append(get_median(sh, sh_invalids, lh, lh_invalids))
    return medians

def get_median(sh, sh_invalids, lh, lh_invalids):
    print()
    print("sh=", sh, sh_invalids)
    print("#### lh=", lh, lh_invalids)
    sh_valid_cnt = len(sh) - sum(sh_invalids.values())
    lh_valid_cnt = len(lh) - sum(lh_invalids.values())
    num_cnt_is_odd = (sh_valid_cnt + lh_valid_cnt) % 2 == 1


    if num_cnt_is_odd:
        median = lh[0]
    else:
        median = (-sh[0] + lh[0]) / 2
    print("****** get_median", median)
    return median



def refresh_sh_top(heap, invalids):
    if 1 <= len(heap) and 1 <= invalids[- heap[0]]:
        out = - heappop(heap)
        invalids[out] -= 1

def refresh_lh_top(heap, invalids):
    print("refresh lh: ", heap, invalids)
    if 1 <= len(heap) and 1 <= invalids[heap[0]]:
        out = heappop(heap)
        invalids[out] -= 1
    print("refreshed lh: ", heap, invalids)
    

# test
# assert sliding_medians([1,3,2,3,6], 3) == [2,3,3]

# # # odd elements in SW
# assert sliding_medians([1, 3], 1) == [1, 3]

# # even elements in SW
# assert sliding_medians([1, 3], 2) == [2]

# assert sliding_medians([1], 2) is None
# assert sliding_medians([1], 1) == [1]
# assert sliding_medians([1], 0) is None 


# assert sliding_medians([1,3,-1,-3,5,3,6,7], 3) == [1.00000,-1.00000,-1.00000,3.00000,5.00000,6.00000]
print(sliding_medians([1,3,-1,-3,5,3,6,7], 3))


# [-1],[1,3] -> add -3, remove 1
# [-3], [-1,3,x1] -> add 5, remove 3
# [-3], [1,x3,5]


sh= [1] defaultdict(<class 'int'>, {})
#### lh= [1, 3] defaultdict(<class 'int'>, {})
****** get_median 1
removed left= 1
added right= -3
refresh lh:  [1, 3] defaultdict(<class 'int'>, {-1: 0, 1: 1})
refreshed lh:  [3] defaultdict(<class 'int'>, {-1: 0, 1: 0})
refresh lh:  [3] defaultdict(<class 'int'>, {-1: 0, 1: 0})
refreshed lh:  [3] defaultdict(<class 'int'>, {-1: 0, 1: 0, 3: 0})
refresh lh:  [-1, 3] defaultdict(<class 'int'>, {-1: 0, 1: 0, 3: 0})
refreshed lh:  [-1, 3] defaultdict(<class 'int'>, {-1: 0, 1: 0, 3: 0})

sh= [3] defaultdict(<class 'int'>, {-1: 0, -3: 0})
#### lh= [-1, 3] defaultdict(<class 'int'>, {-1: 0, 1: 0, 3: 0})
****** get_median -1
removed left= 3
added right= 5
refresh lh:  [-1, 3] defaultdict(<class 'int'>, {-1: 0, 1: 0, 3: 1})
refreshed lh:  [-1, 3] defaultdict(<class 'int'>, {-1: 0, 1: 0, 3: 1})
refresh lh:  [-1, 3, 5] defaultdict(<class 'int'>, {-1: 0, 1: 0, 3: 1})
refreshed lh:  [-1, 3, 5] defaultdict(<class 'int'>, {-1: 0, 1: 0, 3: 1})
refresh lh:  [-1,

In [ ]:
# Minimum Cost to Hire K Workers

'''
# interface
n workers
Args:
    quality: list of len n
    wage   : list of len n
    k: int. extact num of workers to hire
Returns:
    the least amount of money needed to form a paid group satisfying the above conditions.

# example
# algorithm

k = 2
wage = 　　[70,50,30]
quality = [10,20,5] 


wage/q =   7, 2.5, 6

So,
w0, w1 -> max(7, 2.5) * (10 + 20)　= 210
w0, w2 -> max(7, 6) * (10 + 5)　= 105
w0, w2 -> max(2,5, 6) * (20 + 5)　= 150

So return min = 105

time  = N**2
space = N

# algorithm2
sort with wage/q
wage = 　  [50, 30, 70]
quality =  [20, 5, 10] 
wage/q =   2.5, 6,  7
            <---*       = 6 * least 2 quality sum
            <-------*   = 7 * least 2 quality sum


low_k_qualities = [] as max heap of length k

for each wage_per_q,
    push to low_k_qualities
    pop largest
    least_cost = min(low_k_qualities * sum of low_k_qualities)
return least_cost

time  = N * (logK + K) <--------optimze this to N logK (with keepting track of sum of heap)
space = N
'''

def mincostToHireWorkers(self, quality: List[int], wage: List[int], k: int) -> float:
    n = len(wage)
    if n < k:
        return None

    wage_per_quality = []
    for q, w in zip(quality, wage):
        wage_per_quality.append(w/q)
    # print(wage_per_quality)

    
    quality_with_index = [(q, idx) for idx, q in enumerate(quality)]
    quality_with_index.sort(key=lambda q_idx: wage_per_quality[q_idx[1]])
    qualities = [q_idx[0] for q_idx in quality_with_index]
    wage_per_quality.sort()
    # print(qualities, wage_per_quality)

    least_k_qualities = [-q for q in qualities[:k]] # max heap of length k
    lest_k_sum = sum(qualities[:k])
    heapq.heapify(least_k_qualities)
    curr_wpq = wage_per_quality[k-1]
    least_cost = lest_k_sum * curr_wpq
    # print(least_cost)

    for i in range(k, n):
        curr_wpq = wage_per_quality[i]
        heapq.heappush(least_k_qualities, - qualities[i])
        neg_out = heapq.heappop(least_k_qualities)
        lest_k_sum = lest_k_sum + qualities[i] + neg_out
        curr_cost = lest_k_sum * curr_wpq
        least_cost = min(least_cost, curr_cost)
    
    return least_cost

# ================================================================================================
# After reading https://leetcode.com/problems/minimum-cost-to-hire-k-workers/solutions/2121657/python-faster-than-99-memory-86/, 
# I noticed that I can zip quality and wpq.

def mincostToHireWorkers(self, quality: List[int], wage: List[int], k: int) -> float:
    n = len(wage)
    if n < k:
        return None

    # sort by ratio
    wpq_and_quality = [(w/q, q) for q, w in zip(quality, wage)]
    wpq_and_quality.sort()

    # initialize max heap
    low_k_qualities = [-q for _wpq, q in wpq_and_quality[:k]] # used as max heap of length K
    heapq.heapify(low_k_qualities)
    sum_of_low_k = - sum(low_k_qualities)

    # initialize cost
    least_cost = wpq_and_quality[k-1][0] * sum_of_low_k


    # check each ratio - update max heap and cost
    for i in range(k, n):
        curr_wpq, curr_quality = wpq_and_quality[i]

        heapq.heappush(low_k_qualities, - curr_quality)
        neg_q = heapq.heappop(low_k_qualities)
        sum_of_low_k += curr_quality + neg_q
        
        curr_cost = curr_wpq * sum_of_low_k
        least_cost = min(least_cost, curr_cost)
    
    return least_cost

In [ ]:
# Merge k Sorted Lists

'''
# interface
lists:
    array of k liked lists.
        each ll is sorted in ascending order.

# example
head -> 1a -> 4a -> 5a
head -> 1b -> 3b -> 4b
head -> 2c -> 6c

Return
head -> 1 -> 1 -> 2 -> 3 -> 4 -> 4 -> 5 -> 6

# algorithm
Use heap.

[1a, 1b, 2c]
pop 1a from heap. Connect to returned ll.
push 4a into heap

[4a, 1b, 2c]
pop 1b from heap. Connect to returned ll.
push 3b into heap

[4a, x1b, 2c]
[4a, 3b, 2c]
[4a, 3b, 6c]
[4a, 4b, 6c]
[5a, 4b, 6c]
[x , 4b, 6c] --- If next is None, don't push back.

time = N * log K
space = K
'''

# Definition for singly-linked list.
# class ListNode:
#     def __init__(self, val=0, next=None):
#         self.val = val
#         self.next = next
class Solution:
    # head->1->3, head->2
    def mergeKLists(self, lists: List[Optional[ListNode]]) -> Optional[ListNode]:
        h = [] # used as min heap of (val, idx in lists)
        for idx, l in enumerate(lists):
            if l is not None:
                h.append((l.val, idx))
        heapq.heapify(h)
                        #                                    ^curr
        merged = ListNode(None) # pseudo node -> node1 -> node2  -> node3 -> None
        curr = merged
        while 0 < len(h):
            val, idx = heapq.heappop(h) # 1 node1 -> 2, node2 -> 3, node3

            if lists[idx] is None:
                continue
            
            curr.next = lists[idx]
            curr = curr.next 

            lists[idx] = lists[idx].next
            if lists[idx] is not None: # node3
                heapq.heappush(h, (lists[idx].val, idx))

            ################### This has to be after lists[idx] = lists[idx].next
            curr.next = None
        
        return merged.next
